#### Note To Self

As of now, when I run the tectonic earth simulations for too long, the resulting mountains get too big. Although I have made attempts at solving this issue in the past, I feel like erosion processes is what I should be using. I should modify the Gospl notebook so that a new gospl simulation is run after each tectonic earth simulation iteration, and see if the Gospl errosion algorithm prevents my mountains from growing too high.

In [ ]:
import numpy as np
import pyvista as pv
from TectonicEarth import Earth
import matplotlib.pyplot as plt
from scipy.spatial import cKDTree
from GosplManager import GosplManager
from PlateBoundaries import Boundaries
from EarthsAssistant import EarthAssist

In [ ]:
#Code to confirm that all code based on previous notebooks works as expected
if False:
    earth = Earth(
                startTime = 5,
                endTime = 0,
                deltaTime = 1,
                baseUplift = 2000,
                distTransRange = 1000000, 
                numToAverageOver = 10,
                earthRadius = 6378137.,
                useKilometres = False,
                useGospl = True)
    
    #Run earth simulation, run Gospl simulation and animate results
    earth.runTectonicSimulation()
    gosplMan = GosplManager(earth, subdivisions=4)
    gosplMan.createAllFilesAndRunSimulation()
    gosplMan.animateGosplOutput()

# Introduction

As of now, our earth simulation seems to look ok as it runs over shorter periods of time. However, various errors become obvious as we run the simulation for longer. The most prominent problem so far is that when two continents diverge, they don't leave ocean behind. Additionally, we don't have a mechanism for ocean floor formations. These are both phenomina related to tectonic plate divergence which we will work towards solving in this notebook.

This problem can be demonstrated clearly in the resulting animation bellow:

In [ ]:
%%HTML
<video width="800" height="608" controls>
  <source src="./NotebookFiles/DivergenceProblem.mp4" type="video/mp4">
</video>

``` python
#Code for generating the above video
if False:
    earth = Earth(
        startTime = 100,
        endTime = 0,
        deltaTime = 2,
        baseUplift = 2000,
        distTransRange = 1000000, 
        numToAverageOver = 10,
        earthRadius = 6378137.,
        useKilometres = False,
        useGospl = True)
    earth.runTectonicSimulation()
    earth.animate()
```

# Diverging Continents

In order to make diverging contents leave oceans behind, we need to lower vertices that are nearby diverging plate boundaries. In our code, we will refer to the amount of lowering as *divergeLowering*, which will depend on the following transfers:

- **Distance Transfer** - The distance that a vertex is to the closest diverging plate boundary. Vertices that are close to a plate boundary should be lowered more than vertices far away from the boundaries, and so we will pass the distances through a gaussian shaped function to calculate the distance contribution to diverge lowering. Given the mean $\mu$ and variance $\sigma$, our gaussian is defined by:

$$
Gaussian(x, \mu, \sigma) = exp \Big( \frac{- (x - \mu)^2}{\sigma} \Big)
$$

- **Height Transfer** - The current height of the vertex. Vertices that are on a higher elevation (on land) should be lowered more than vertices that have a lower elevation (in ocean). Therefore the height contribution will be calculated by passing the current heights of vertices through a sigmoid function. Given the centre $\mu$ and steepness $s$, our sigmoid is defined by:

$$
Sigmoid(x, \mu, s) = \frac{1}{1 + e^{- s (x - \mu)}}
$$

In the code bellow, we define these two functions and provide a simple plot of each. Note that the default parameters are chosen so that most of the function outputs varies at input values in the range $x \in [-1, 1]$.

In [ ]:
#Gaussian function used for profile of diverge lowering
def gaussian(x, mean=0, variance=0.25):
    return np.exp(-((x - mean)**2) / variance)

#Sigmoid function
def sigmoid(x, centre=-0.1, steepness=6):
    return 1 / (1 + np.exp(-(x - centre) * steepness))

#Plot the gaussian and sigmoid functions
x = np.arange(-2, 2, 0.01)
fig, axs = plt.subplots(1, 2, figsize=(15, 6))
axs[0].plot(x, gaussian(x, mean=0, variance=0.25))
axs[0].title.set_text("Gaussian")
axs[0].set_xlabel('Distance Input')
axs[0].set_ylabel('Output')
axs[1].plot(x, sigmoid(x, centre=0, steepness=4))
axs[1].title.set_text("Sigmoid")
axs[1].set_xlabel('Height Input')
axs[1].set_ylabel('Output')

Now that we have define the transfer functions for diverge lowering, we need to get the appropriate heights and distance input values. Getting the heights input is easy, since all we have to do is read the current heights from *Earth.heights*. However, to get the distances from diverging plate boundaries, we first need to get the location of all diverging plate boundaries, which can be identified by having a negative collision speed (as discussed in notebook 2). The distances will then be calculated based on the boundary lines. Most of the code for this is already contained in the *PlateBoundaries.py* file.

The code bellow demonstrates how we will get our distances to diverging plate boundaries.

In [ ]:
#Get coordinates and line points of all diverging plate boundary locations
def getDivergingBoundaries(boundaries):
    divXYZ, divLinePoints = [], []
    for bound in boundaries.plateBoundaries:
        for i in range(bound.lineCentres.shape[0]):
            if bound.collisionSpeed[i] < 0:
                divXYZ.append(bound.lineCentres[i])
                divLinePoints.append(bound.linePoints[i])
    return np.array(divXYZ), np.array(divLinePoints)

#Get distance from vertices to diverging plate boundaries.
def getDistanceToDivergenge(divXYZ, sphereXYZ, linePoints):
    distIds = cKDTree(divXYZ).query(sphereXYZ, k=1)[1]
    distIds[distIds >= divXYZ.shape[0]] = divXYZ.shape[0]-1
    closestLinePoints = linePoints[distIds]
    distToBound = Boundaries.getDistsToLinesSeg(sphereXYZ, closestLinePoints)
    return distToBound

#Create an earth object
earth = Earth(
    startTime = 10,
    endTime = 0,
    deltaTime = 2,
    baseUplift = 2000,
    distTransRange = 1000000, 
    numToAverageOver = 10,
    earthRadius = 6378137.,
    useKilometres = False,
    useGospl = True)

#Create an instance of the Boundaries class (this code is normally called by earth.runTectonicSimulation())
time = earth.startTime - earth.deltaTime
plateIds = earth.getPlateIdsAtTime(time)
rotations = earth.getRotations(plateIds, time)
boundaries = Boundaries(time, earth, plateIds, rotations)

#Run the newly defined functions
divXYZ, divLinePoints = getDivergingBoundaries(boundaries)
distToDivs = getDistanceToDivergenge(divXYZ, earth.sphereXYZ, divLinePoints)

#Visualize the results
plotter = pv.PlotterITK()
earthMesh = earth.getEarthMesh()
earthMesh['distToDivs'] = distToDivs**0.25
boundaryLines = Boundaries.getBoundaryLines(boundaries.plateBoundaries)
plotter.add_mesh(earthMesh, scalars='distToDivs')
plotter.add_mesh(boundaryLines, color='r')
plotter.show()

Using the data that we have just calculated, getting the diverge lowering is simple, as shown bellow:

In [ ]:
#Define function for diverge lowering
def getDivergeLowering(distToDivs, heights, maxDistance=200000, minMaxHeights=8000):
    distanceTransfer = gaussian(distToDivs / maxDistance)
    heightTransfer = sigmoid(heights / minMaxHeights)
    return distanceTransfer * heightTransfer

#Get diverge lowering
divergeLowering = getDivergeLowering(distToDivs, earth.heights)

#Visualize results
plotter = pv.PlotterITK()
earthMesh['divergeLowering'] = divergeLowering
plotter.add_mesh(earthMesh, scalars='divergeLowering')
plotter.add_mesh(boundaryLines, color='r')
plotter.show()

Now that we have gotten the diverge lowering property, we want to apply it at each of earth's simulation step. In the code bellow, the function *doSimulationStep()* is similar to *earth.doSimulationStep()*, but with diverge lowering also implemented in it. We then provide an animation of the results.

In [ ]:
#Given an earth object, apply diverge lowering to it
def applyDivergeLowering(earth, baseLowering=2000):
    divXYZ, divLinePoints = getDivergingBoundaries(earth.boundaries)
    distToDivs = getDistanceToDivergenge(divXYZ, earth.sphereXYZ, divLinePoints)
    earth.heights -= baseLowering * getDivergeLowering(distToDivs, earth.heights)

#Function that is similar to earth.doSimulationStep() but with added code from this notebook
def doSimulationStep(earth, time):
    earth.heights = np.copy(earth.heightHistory[-1])
    earth.timeHistory.append(time - earth.deltaTime)
    plateIds = earth.getPlateIdsAtTime(time)
    rotations = earth.getRotations(plateIds, time)
    earth.doSubductionUplift(time, plateIds, rotations)
    applyDivergeLowering(earth) #New line of code to be added in earth.doSimulationStep()
    earth.movePlatesAndRemesh(plateIds, rotations)
    earth.createTectonicDisplacements()
    
#Create new earth object
earth = Earth(
    startTime = 100,
    endTime = 80,
    deltaTime = 2,
    baseUplift = 2000,
    distTransRange = 1000000, 
    numToAverageOver = 10,
    earthRadius = 6378137.,
    useKilometres = False,
    useGospl = True)

#Lopp that is similar to earth.runTectonicSimulation()
for time in earth.simulationTimes[:-1]:
    print('Currently simulating at {} Millions years ago'.format(time), end='\r')
    doSimulationStep(earth, time)

earth.animate(lookAtLonLat=[0, 0])

In [ ]:
%%HTML
<video width="800" height="608" controls>
  <source src="./TectonicSimulation.mp4" type="video/mp4">
</video>

In [ ]:





#Get a map of where and how much to lower vertices due to continental rifting
def getContinentalLowering(riftXYZ, sphereXYZ, linePoints, gaussVariance=20000000000):
    distIds = cKDTree(riftXYZ).query(sphereXYZ)[1]
    closestLinePoints = linePoints[distIds]
    distToBound = Boundaries.getDistsToLinesSeg(sphereXYZ, closestLinePoints)
    divergeLowering = gaussian(distToBound, variance=gaussVariance)
    return distToBound, divergeLowering

#Run the newly created functions
continentalRifts, riftXYZ, linePoints = getContinentalRiftData(boundaries)
distToBound, divergeLowering = getContinentalLowering(riftXYZ, earth.sphereXYZ, linePoints)
divergeLoweringWithoutSigmoid = np.copy(divergeLowering)
divergeLowering *= sigmoid(earth.heights / 10000, centre=-0.1, steepness=6)

#Create earthMesh object for plotting
earthMesh = earth.getEarthMesh()
earthMesh['divergeLoweringWithoutSigmoid'] = divergeLoweringWithoutSigmoid
earthMesh['divergeLowering'] = divergeLowering
earthMesh['distToBound'] = distToBound

#Visualize the results
plotter = pv.PlotterITK()
plotter.add_mesh(earthMesh, scalars='divergeLowering')
plotter.show()

In [ ]:
earth = Earth(
    startTime = 100,
    endTime = 80,
    deltaTime = 2,
    baseUplift = 2000,
    distTransRange = 1000000, 
    numToAverageOver = 10,
    earthRadius = 6378137.,
    useKilometres = False,
    useGospl = True)

time = earth.startTime - earth.deltaTime
plateIds = earth.getPlateIdsAtTime(time)
rotations = earth.getRotations(plateIds, time)
boundaries = Boundaries(time, earth, plateIds, rotations)

sphereXYZ = earth.sphereXYZ

kdTreeOfEachPlate = {}
vertexIds = {}
for idx in np.unique(plateIds):
    if sphereXYZ[plateIds==idx].shape[0] == 0:
        continue
    kdTreeOfEachPlate[idx] = cKDTree(sphereXYZ[plateIds==idx])
    vertexIds[idx] = np.argwhere(plateIds==idx)
    #print(vertexIds[idx])

for bound in boundaries.plateBoundaries:
    sharedIds = bound.sharedPlateIds
    boundXYZ = bound.XYZ
    for i in range(boundXYZ.shape[0]):
        leftId = sharedIds[i, 0]
        rightId = sharedIds[i, 1]
        XYZ = boundXYZ[i]
        closestLeftVertexID = kdTreeOfEachPlate[leftId].query(XYZ)[1]
        closestRightVertexID = kdTreeOfEachPlate[rightId].query(XYZ)[1]
        closestLeftVertex = sphereXYZ[vertexIds[leftId][closestLeftVertexID]]
        closestrightVertex = sphereXYZ[vertexIds[rightId][closestRightVertexID]]
        
        #print(closestLeftVertex)
        #print(closestrightVertex)
        #print()
        
        
    #print(bound.sharedPlateIds.shape)
    

#### Get distances to diverging plate boundaries.

The code bellow demonstrates how we will implement diverging continents. Although the algorithm itself is working fine, I probably still need to improve the identification of diverging plate boundaries.

In [ ]:
earth = Earth(
    startTime = 100,
    endTime = 80,
    deltaTime = 2,
    baseUplift = 2000,
    distTransRange = 1000000, 
    numToAverageOver = 10,
    earthRadius = 6378137.,
    useKilometres = False,
    useGospl = True)

#earth.doSimulationStep(earth.startTime)

time = earth.startTime - earth.deltaTime
plateIds = earth.getPlateIdsAtTime(time)
rotations = earth.getRotations(plateIds, time)
boundaries = Boundaries(time, earth, plateIds, rotations)

In [ ]:
#Gaussian function use for profile of diverge lowering
def gaussian(x, mean=0, variance=0.25):
    return np.exp(-((x - mean)**2) / variance)

def sigmoid(x, centre=0, steepness=1):
    return 1 / (1 + np.exp(-(x - centre) * steepness))
'''
#Get relevant data for continental rifts
def getContinentalRiftData(boundaries):
    continentalRifts, riftXYZ, linePoints = [], [], []
    for bound in boundaries.plateBoundaries:
        if (bound.gpmlBoundType == 'gpml:ContinentalRift'):
            for i in range(bound.lineCentres.shape[0]):
                riftXYZ.append(bound.lineCentres[i])
                linePoints.append(bound.linePoints[i])
            continentalRifts.append(bound)
    return continentalRifts, np.array(riftXYZ), np.array(linePoints)
'''
#Get relevant data for continental rifts
def getContinentalRiftData(boundaries):
    continentalRifts, riftXYZ, linePoints = [], [], []
    for bound in boundaries.plateBoundaries:
        for i in range(bound.lineCentres.shape[0]):
            if bound.collisionSpeed[i] < 0:
                riftXYZ.append(bound.lineCentres[i])
                linePoints.append(bound.linePoints[i])
        continentalRifts.append(bound)
    return continentalRifts, np.array(riftXYZ), np.array(linePoints)

#Get a map of where and how much to lower vertices due to continental rifting
def getContinentalLowering(riftXYZ, sphereXYZ, linePoints, gaussVariance=20000000000):
    distIds = cKDTree(riftXYZ).query(sphereXYZ)[1]
    closestLinePoints = linePoints[distIds]
    distToBound = Boundaries.getDistsToLinesSeg(sphereXYZ, closestLinePoints)
    divergeLowering = gaussian(distToBound, variance=gaussVariance)
    return distToBound, divergeLowering

#Run the newly created functions
continentalRifts, riftXYZ, linePoints = getContinentalRiftData(boundaries)
distToBound, divergeLowering = getContinentalLowering(riftXYZ, earth.sphereXYZ, linePoints)
divergeLoweringWithoutSigmoid = np.copy(divergeLowering)
divergeLowering *= sigmoid(earth.heights / 10000, centre=-0.1, steepness=6)

#Create earthMesh object for plotting
earthMesh = earth.getEarthMesh()
earthMesh['divergeLoweringWithoutSigmoid'] = divergeLoweringWithoutSigmoid
earthMesh['divergeLowering'] = divergeLowering
earthMesh['distToBound'] = distToBound

#Visualize the results
plotter = pv.PlotterITK()
plotter.add_mesh(earthMesh, scalars='divergeLowering')
plotter.show()

In [ ]:
#Function that is similar to earth.doSimulationStep but with added code from this notebook
def doSimulationStep(earth, time):
    earth.heights = np.copy(earth.heightHistory[-1])
    earth.timeHistory.append(time - earth.deltaTime)
    plateIds = earth.getPlateIdsAtTime(time)
    rotations = earth.getRotations(plateIds, time)
    earth.doSubductionUplift(time, plateIds, rotations)
    
    #Newly added functionality
    boundaries = Boundaries(time, earth, plateIds, rotations)
    continentalRifts, riftXYZ, linePoints = getContinentalRiftData(boundaries)
    distToBound, divergeLowering = getContinentalLowering(riftXYZ, earth.sphereXYZ, linePoints)
    divergeLowering *= sigmoid(earth.heights / 10000, centre=-0.1, steepness=6)
    earth.heights -= divergeLowering * 2000
    
    earth.movePlatesAndRemesh(plateIds, rotations)
    earth.createTectonicDisplacements()
    
    

earth = Earth(
    startTime = 100,
    endTime = 80,
    deltaTime = 2,
    baseUplift = 2000,
    distTransRange = 1000000, 
    numToAverageOver = 10,
    earthRadius = 6378137.,
    useKilometres = False,
    useGospl = True)

#Lopp that is similar to earth.runTectonicSimulation()
for time in earth.simulationTimes[:-1]:
    print('Currently simulating at {} Millions years ago'.format(time), end='\r')
    doSimulationStep(earth, time)

In [ ]:
earth.animate(lookAtLonLat=[0, 0])

In [ ]:
def sigmoid(x, centre=0, steepness=1):
    return 1 / (1 + np.exp(-(x - centre) * steepness))

xRange = 20000
x = np.arange(-xRange, xRange, xRange/1000)
y = sigmoid(x / 10000, centre=-0.1, steepness=6)
plt.plot(x, y)
plt.grid()
plt.show()

In [ ]:
#heightHist = np.array(earth.heightHistory)
#np.savez_compressed('heightHistorySaves/1', heightHist= heightHist)

In [ ]:
print(earth.heights)
print(np.max(earth.heights))
print(np.min(earth.heights))

In [ ]:
'''
includeBoundariesOfType = [
    #'gpml:ContinentalRift', #Should be included
    #'gpml:Fault', #Dont include this (planar fracture)
    #'gpml:MidOceanRidge', #Diverge but should be considered in ocean floor generation
    'gpml:FractureZone', 
    'gpml:InferredPaleoBoundary',
    #'gpml:Transform' #When plates slide against each other
]
'''

includeBoundariesOfType = [
    'gpml:ContinentalRift', #Should definetely be included
    #'gpml:Fault', #Dont include this (planar fracture)
    #'gpml:FractureZone', #Plates sliding against each other
    #'gpml:InferredPaleoBoundary', #Plates sliding against each other
    #'gpml:MidOceanRidge', #Dont include Diverge but should be considered in ocean floor generation
    #'gpml:OrogenicBelt', #Don't include this
    #'gpml:SubductionZone', #Don't include this
    #'gpml:TerraneBoundary', #Don't include
    #'gpml:Transform',  #Dont include - When plates slide against each other (don't include)
    #'gpml:UnclassifiedFeature' #Don't include this
]
boundTypes = []
divergingBoundaries = []
for bound in boundaries.plateBoundaries:    
    for boundType in includeBoundariesOfType:
        if (bound.gpmlBoundType == boundType):
            divergingBoundaries.append(bound)
    boundTypes.append(bound.gpmlBoundType)
print(np.unique(boundTypes))

linePoints, boundXYZ = [], []
for bound in divergingBoundaries:
    for i in range(bound.lineCentres.shape[0]):
        boundXYZ.append(bound.lineCentres[i])
        linePoints.append(bound.linePoints[i])

linePoints = np.array(linePoints)
boundXYZ = np.array(boundXYZ)
print(linePoints.shape)

In [ ]:


xyzCount = 0
linesXYZ, lineConnectivity = [], []
for bound in divergingBoundaries:
    numOfPoints = len(bound.XYZ)
    lineConnectivity.append(numOfPoints)
    lineID = np.arange(numOfPoints) + xyzCount

    #Loop through points in plate boundary and append to arrays
    for i in range(numOfPoints):
        lineConnectivity.append(lineID[i])
        linesXYZ.append(bound.XYZ[i])
    xyzCount += numOfPoints

lineMesh = pv.PolyData(np.array(linesXYZ) * 1.01, lines=lineConnectivity)


sphereXYZ = earth.sphereXYZ
distIds = cKDTree(boundXYZ).query(sphereXYZ)[1]
closestLinePoints = linePoints[distIds]
distToBound = Boundaries.getDistsToLinesSeg(sphereXYZ, closestLinePoints)
divergeLowering = gaussian(distToBound / 200000)


earthMesh = earth.getEarthMesh()
earthMesh['divergeLowering'] = divergeLowering
earthMesh['distToBound'] = distToBound**0.25

tectonicDisp = earth.tectonicDisplacementHistory[-1]
tectonicDispMesh = EarthAssist.createTectonicVectorMesh(sphereXYZ, tectonicDisp, ignoreEveryNverts=2)

plotter = pv.PlotterITK()
#plotter.add_mesh(earthMesh, scalars=distToBound**0.25)
plotter.add_mesh(earthMesh, scalars='heights')
plotter.add_mesh(lineMesh)
plotter.add_mesh(tectonicDispMesh)
plotter.show(window_size=[800, 400])